## import


In [44]:
import pandas as pd
import mysql.connector
from sqlalchemy.sql import text
from sqlalchemy import create_engine, MetaData, URL, Identity
from sqlalchemy import Table, Column, Integer, String, BOOLEAN, DATE, ForeignKey, TEXT

# connect to database

In [45]:
meta = MetaData()
USERNAME = 'root'
PASSWORD = 'A1382L1234i@#'
SERVER = 'localhost'
PORT = 3306
DATABASE = 'book-store'

url_object = URL.create("mysql+mysqlconnector",
                        username=USERNAME,
                        password=PASSWORD,
                        host=SERVER,
                        database=DATABASE)
engine = create_engine(url_object, echo=False)
conn = engine.connect()

## Creating tables and Inserting Data

### Book

In [46]:
book = Table(
    'book', meta,
    Column('code', Integer, primary_key=True),
    Column('title_persian', String(200)),
    Column('title_english', String(200)),
    Column('price', Integer),
    Column('discount', Integer),
    Column('grade', Integer),
    Column('shabak', String(2000)),
    Column('page_count', Integer),
    Column('release_year_sh', Integer),
    Column('release_year_mi', Integer),
    Column('exist', BOOLEAN),
    Column('earliest_access', DATE),
    Column('print_series', Integer),
    Column('cover', String(50)),
    Column('ghate', String(50))
)
meta.create_all(engine)


In [47]:
df_book = pd.read_csv('book.csv')
df_book.dropna(subset=["code"],inplace=True)
df_book.drop_duplicates(subset=["code"],inplace=True)
df_book.to_csv('modified/modified_book.csv', index=False, encoding='utf-8-sig')
df_book.to_sql('book', engine, if_exists='append', index=False)

94389

### Group

In [48]:
group = Table(
    'group', meta,
    Column('id', Integer, primary_key=True),
    Column('description', TEXT)
)
meta.create_all(engine)


In [49]:
df_group = pd.read_csv('group.csv')
df_group.dropna(subset=["group_id"],inplace=True)
df_group.drop_duplicates(subset=["group_id"],inplace=True)
df_group['group_id'] = df_group['group_id'].astype(int).astype(object)
df_group.rename(columns={'group_id': 'id'}, inplace=True)
if df_group["description"].isnull().all():
    df_group["description"] = 'None'
df_group.to_csv('modified/modified_group.csv', index=False, encoding='utf-8-sig')


In [50]:
df_group.head(50000).to_sql('group', engine, if_exists='append', index=False)
df_group.tail(len(df_group['id'].values.tolist()) - 50000).to_sql('group', engine, if_exists='append', index=False)

34787

### Group_Book

In [51]:
group_book = Table(
    'group_book', meta,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('group_id', Integer, ForeignKey('group.id')),
    Column('book_code', Integer, ForeignKey('book.code'))
)
meta.create_all(engine)


In [52]:
df_group_book = pd.read_csv('group_book.csv')
df_group_book.dropna(subset=["group_id"],inplace=True)
df_group_book.dropna(subset=["book_code"],inplace=True)
df_group_book['book_code'] = df_group_book['book_code'].astype(int)
for index in df_group_book.index:
    if ( df_group_book.loc[index, 'book_code'] not in df_book['code'].values.tolist()) or (df_group_book.loc[index, 'group_id'] not in df_group['id'].values.tolist()):
        df_group_book.drop(index=index, inplace=True)
df_group_book.to_csv('modified/modified_group_book.csv', index=False, encoding='utf-8-sig')
df_group_book.to_sql('group_book', engine, if_exists='append', index=False)

87929

### Person

In [53]:
person = Table(
    'person', meta,
    Column('counter', Integer, primary_key=True, autoincrement=False),
    Column('person_id', Integer),
    Column('name', String(100)),
    Column('description', TEXT)
)
meta.create_all(engine)


In [54]:
df_person = pd.read_csv('person.csv')
df_person.dropna(subset=["counter"],inplace=True)
df_person.dropna(subset=["name"], inplace=True)
df_person.drop_duplicates(subset=["counter"],inplace=True)
df_person.rename(columns={'id': 'person_id'}, inplace=True)
# if df_person["description"].isnull().all():
#     df_person["description"] = 'None'
df_person.to_csv('modified/modified_person.csv', index=False, encoding='utf-8-sig')
df_person.to_sql('person', engine, if_exists='append', index=False)

50849

### Publisher

In [55]:
publisher = Table(
    'publisher', meta,
    Column('id', Integer, primary_key=True),
    Column('name', String(100))
)
meta.create_all(engine)


In [56]:
df_publisher = pd.read_csv('publisher.csv')
df_publisher.dropna(subset=["id"],inplace=True)
df_publisher.drop_duplicates(subset=["id"],inplace=True)
df_publisher.to_csv('modified/modified_publisher.csv', index=False, encoding='utf-8-sig')
df_publisher.to_sql('publisher', engine, if_exists='append', index=False)

2098

### Category

In [57]:
category = Table(
    'category', meta,
    Column('id', Integer, primary_key=True),
    Column('name', String(150)),
    Column('description', TEXT)
)
meta.create_all(engine)


In [58]:
df_category = pd.read_csv('category.csv')
df_category.dropna(subset=["id"],inplace=True)
df_category.drop_duplicates(subset=["id"],inplace=True)
if df_category["description"].isnull().all():
    df_category["description"] = 'None'
df_category.to_csv('modified/modified_category.csv', index=False, encoding='utf-8-sig')
df_category.to_sql('category', engine, if_exists='append', index=False)

1078

### Group_Category

In [59]:
group_category = Table(
    'group_category', meta,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('group_id', Integer, ForeignKey('group.id')),
    Column('category_id', Integer, ForeignKey('category.id'))
)
meta.create_all(engine)


In [60]:
df_group_category = pd.read_csv('group_category.csv')
df_group_category.dropna(subset=["group_id"],inplace=True)
df_group_category.dropna(subset=["category_id"],inplace=True)
for index in df_group_category.index:
    if (df_group_category.loc[index, 'group_id'] not in df_group['id'].values.tolist()) or (df_group_category.loc[index, 'category_id'] not in df_category['id'].values.tolist()):
        df_group_category.drop(index=index, inplace=True)
df_group_category.to_csv('modified/modified_group_category.csv', index=False, encoding='utf-8-sig')
df_group_category.to_sql('group_category', engine, if_exists='append', index=False)

404581

### Book_publisher

In [61]:
book_publisher = Table(
    'book_publisher', meta,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('book_code', Integer, ForeignKey('book.code')),
    Column('publisher_id', Integer, ForeignKey('publisher.id'))
)
meta.create_all(engine)


In [62]:
df_book_publisher = pd.read_csv('book_publisher.csv')
df_book_publisher.dropna(subset=["publisher_id"],inplace=True)
df_book_publisher.dropna(subset=["book_code"],inplace=True)
df_book_publisher['book_code'] = df_book_publisher['book_code'].astype(int)
df_book_publisher.to_csv('modified/modified_book_publisher.csv', index=False, encoding='utf-8-sig')
for index in df_book_publisher.index:
    if (df_book_publisher.loc[index, 'book_code'] not in df_book['code'].values.tolist()) or (df_book_publisher.loc[index, 'publisher_id'] not in df_publisher['id'].values.tolist()):
        df_book_publisher.drop(index=index, inplace=True)
df_book_publisher.to_sql('book_publisher', engine, if_exists='append', index=False)

86335

### Crew

In [63]:
crew = Table(
    'crew', meta,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('book_code', Integer, ForeignKey('book.code')),
    Column('person_counter', Integer, ForeignKey('person.counter')),
    Column('role', String(15))
)
meta.create_all(engine)


In [64]:
df_crew = pd.read_csv('crew.csv')
df_crew.dropna(subset=["book_code"],inplace=True)
df_crew.dropna(subset=["person_counter"],inplace=True)
df_crew.dropna(subset=["role"],inplace=True)
df_crew['person_counter'] = df_crew['person_counter'].astype('int')
for index in df_crew.index:
    if (df_crew.loc[index, 'book_code'] not in df_book['code'].values.tolist()) or (df_crew.loc[index, 'person_counter'] not in df_person['counter'].values.tolist()):
        df_crew.drop(index=index, inplace=True)
df_crew.to_csv('modified/modified_crew.csv', index=False, encoding='utf-8-sig')
df_crew.to_sql('crew', engine, if_exists='append', index=False)

134007